In [18]:
import datetime
import unittest
import ipynb.fs.full.M3_B2_PROJECT_3 as pj3
import pandas as pd
from tabulate import tabulate 
from dateutil.relativedelta import relativedelta
import numpy as np

In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [20]:
def splitDate(dateStr):
    dateArr = dateStr.split(" ")
    return dateArr

In [21]:
def getMonth(monthStr):
    full_date = datetime.datetime.strptime(monthStr, "%b")
    return full_date.month

In [22]:
# Function to compare Dates 
#Enter later date first(date1) and Earlier date second(date2)
def compareDate(date1,date2):
    if date1 == 'NA' or date2 == 'NA':
        return True
    else: 
        date_1 = splitDate(date1)
        date_2 = splitDate(date2)
        comparedate1 = datetime.date(int(date_1[0]), int(getMonth(date_1[1])), int(date_1[2]))
        comparedate2 = datetime.date(int(date_2[0]), int(getMonth(date_2[1])), int(date_2[2]))
        compare = comparedate2 < comparedate1
        if (compare == False):
            return False;
        else:
            return True

In [23]:
def agecalc(birthdate, deathdate,t):
    if birthdate == 'NA' or deathdate == 'NA':
        return 0
    else:
        birth_date = splitDate(birthdate)
        death_date = splitDate(deathdate)
        birth_date = datetime.date(int(birth_date[0]), int(getMonth(birth_date[1])), int(birth_date[2]))
        death_date = datetime.date(int(death_date[0]), int(getMonth(death_date[1])), int(death_date[2]))
        if t == 'y':
            diff = relativedelta(death_date,birth_date).years
        if t == 'm':
            diff = relativedelta(death_date,birth_date).months
    return diff


In [24]:
# Story US01 Task T01.03
#Check the Individual table for Birth dates and Death dates 
def story_us01_refactor(df_indi,df_fam): 
    now = datetime.datetime.now()
    today = now.strftime("%Y %b %d")      
    df_indi['Date Check_b'] = df_indi.apply(lambda row : compareDate(today,row['Birth Date']),axis = 1)
    df_indi['Date Check_d'] = df_indi.apply(lambda row : compareDate(today,row['Death Date']),axis = 1)
    for i in range(len(df_indi)):
        if str(df_indi.loc[i,'Date Check_b']) == 'False':
            print("ERROR: Individual ID ",df_indi.loc[i,'ID'],df_indi.loc[i,'Birth Date'], "has Date of Birth in Future" )
        else: print("Record",df_indi.loc[i,'ID']," Check OK for Birth Dates ")

# Story US01 Task T01.04 
        if str(df_indi.loc[i,'Date Check_d']) == 'False':
            print("ERROR: Individual ID ",df_indi.loc[i,'ID'],df_indi.loc[i,'Death Date'], "has Date of Death in Future" )
        else: print("Record",df_indi.loc[i,'ID'],"Check OK for Death Dates ")
# Story US01 Task T01.05
# Check Marriage Dates in Family table 
    df_fam['Date Check_b'] = df_fam.apply(lambda row : compareDate(today,row['Married']),axis = 1)
    for i in range(len(df_fam)):
        if str(df_fam.loc[i,'Date Check_b']) == 'False':
            print("ERROR:","Family ID",df_fam.loc[i,"ID"],df_fam.loc[i,"Marriage Date"], "has Date of Wedding in Future" )
        else: print("Record:","Family ID",df_fam.loc[i,"ID"],"Check OK for Wedding Dates ")

In [25]:
# Story US02 
# Birth before Marriage
def story_us02(df_indi,df_fam):
# Task-T02.02 List out Individuals who are married 
    df_husband = df_fam.loc[:,['ID','Married','Husband ID',"Husband's Name"]]
    # Task-T02.03 List out dates for marriage for individuals 
    df_husband = df_husband.merge(df_indi.loc[:,['ID','Birth Date']],
                                  left_on='Husband ID',right_on='ID',
                                  how='inner',suffixes=('_Family','_Individual'))
    # Task-T02.04 Compare birth and death rates 
    df_husband['Date Check'] = df_husband.apply(lambda row : compareDate(row['Married'],row['Birth Date']),axis = 1)
    
    df_wife = df_fam.loc[:,['ID','Married','Wife ID',"Wife's Name"]]
    df_wife = df_wife.merge(df_indi.loc[:,['ID','Birth Date']],
                                  left_on='Wife ID',right_on='ID',
                                  how='inner',suffixes=('_Family','_Individual'))
    # Task-T02.04 Compare birth and death rates 
    df_wife['Date Check'] = df_wife.apply(lambda row : compareDate(row['Married'],row['Birth Date']),axis = 1)
    
    print('Records OK for Marriage Dates')
    df_result_husband = df_husband[df_husband['Date Check']==True]
    print(tabulate(df_result_husband.loc[:,['ID_Family','Married','Husband ID',"Husband's Name", 'Birth Date']],
                   headers = 'keys', tablefmt = 'psql'))
    
    df_result_wife = df_wife[df_wife['Date Check']==True]
    print(tabulate(df_result_wife.loc[:,['ID_Family','Married','Wife ID',"Wife's Name", 'Birth Date']],
                   headers = 'keys', tablefmt = 'psql'))
    
    print("Records ERROR for Marriage Dates ")
    df_result_husband_e = df_husband[df_husband['Date Check']==False]
    print(tabulate(df_result_husband_e.loc[:,['ID_Family','Married','Husband ID',"Husband's Name", 'Birth Date']], 
                   headers = 'keys', tablefmt = 'psql'))
    df_result_wife_e = df_wife[df_wife['Date Check']==False]
    print(tabulate(df_result_wife_e.loc[:,['ID_Family','Married','Wife ID',"Wife's Name", 'Birth Date']],
                   headers = 'keys', tablefmt = 'psql'))

In [26]:
#Story US03 - Birth before death
def us03(df_indiTable):
    df_indi_d = df_indiTable.loc[(df_indiTable["Alive"]==False)] 
    if len(df_indi_d) == 0 :
        print("No Death in family tree")
    else:
        df_indi_d['Date Check'] = df_indi_d.apply(lambda row : compareDate(row['Death Date'],row['Birth Date']),axis = 1)
        df_check_ok = df_indi_d[df_indi_d['Date Check']==True]
        df_check_error = df_indi_d[df_indi_d['Date Check']==False]
        print('Records OK for Date of Death ')
        print(tabulate(df_check_ok.loc[:,df_indiTable.columns],
            headers = 'keys', tablefmt = 'psql'))
        if len(df_check_error) > 0:
            print('Records ERROR for Date of Death ')
            print(tabulate(df_check_error.loc[:,df_indiTable.columns],
                headers = 'keys', tablefmt = 'psql'))
        else:
            print('No errors for Date of Death check')  

In [27]:
#Story US04 - Marriage before divorce
def us04(df_famTable):
    df_divorce = df_famTable.loc[(df_famTable["Divorced"] !='NA')] 
    df_divorce['Date Check'] = df_divorce.apply(lambda row : compareDate(row['Divorced'],row['Married']),axis = 1)
    df_check_ok = df_divorce[df_divorce['Date Check']==True]
    df_check_error = df_divorce[df_divorce['Date Check']==False]
    print('Records OK for Date of Divorce ')
    print(tabulate(df_check_ok.loc[:,df_famTable.columns],
        headers = 'keys', tablefmt = 'psql'))
    if len(df_check_error) > 0:
        print('Records ERROR for Date of Divorce, Date of Divorce occurs before Marriage')
        print(tabulate(df_check_error.loc[:,df_famTable.columns],
            headers = 'keys', tablefmt = 'psql'))
    else:
        print('No errors for Date of Divorce check')

In [28]:
#Story US05 - Marriage before death
def us05(df_indiTable,df_famTable):

    df_result = df_famTable.merge(df_indiTable.loc[:,['ID','Death Date']],
                                    left_on='Husband ID',right_on='ID',
                                    how='left',suffixes=('','_H'))
    
    # print(df_IndiTable)
    df_result = df_result.merge(df_indiTable.loc[:,['ID','Death Date']],
                                    left_on='Wife ID',right_on='ID',
                                    how='left',suffixes=('','_W'))                              
    df_result['Husband Date check'] = df_result.apply(lambda row : compareDate(row['Death Date'],row['Married']), axis=1)
    df_result['Wife Date check'] = df_result.apply(lambda row : compareDate(row['Death Date_W'],row['Married']), axis=1)
 
    for i in range(len(df_result)):
        if str(df_result.loc[i,'Husband Date check']) == 'False':
            print("ERROR : Individual ID",str(df_result.loc[i,'Husband ID']),"has wedding dates later than Date of Death")
        elif str(df_result.loc[i,'Wife Date check']) == 'False':
            print("ERROR : Individual ID",str(df_result.loc[i,'Wife ID']),"has wedding dates later than Date of death")
        else:
            print("Family ID",str(df_result.loc[i,'ID']), "has no wedding dates after Date of Death ")
    

In [29]:
#Story US06 - Divorce before death
def us06(df_indiTable,df_famTable):
    df_famTable_div = df_famTable[df_famTable['Divorced'] != 'NA']
    df_result = df_famTable_div.merge(df_indiTable.loc[:,['ID','Death Date']],
                                    left_on='Husband ID',right_on='ID',
                                    how='left',suffixes=('','_H'))
    
    # print(df_IndiTable)
    df_result = df_result.merge(df_indiTable.loc[:,['ID','Death Date']],
                                    left_on='Wife ID',right_on='ID',
                                    how='left',suffixes=('','_W'))    
                                                       
    df_result['Husband Date check'] = df_result.apply(lambda row : compareDate(row['Death Date'],row['Divorced']), axis=1)
    df_result['Wife Date check'] = df_result.apply(lambda row : compareDate(row['Death Date_W'],row['Divorced']), axis=1)
    for i in range(len(df_result)):
        if str(df_result.loc[i,'Husband Date check']) == 'False':
            print("ERROR : Individual ID",str(df_result.loc[i,'Husband ID']),"has Divorce dates later than Date of Death")
        elif str(df_result.loc[i,'Wife Date check']) == 'False':
            print("ERROR : Individual ID",str(df_result.loc[i,'Wife ID']),"has Divorce dates later than Date of Death")
        else:
            print("Family ID",str(df_result.loc[i,'ID']), "has no Divorce dates after date of Death ")
    

In [30]:
def us07(df_indiTable):
    now = datetime.datetime.now()
    today = now.strftime("%Y %b %d") 
    df_indi_a = df_indiTable.loc[df_indiTable['Alive']==True]
    df_indi_a['Age'] = df_indi_a.apply(lambda row: agecalc(row["Birth Date"],today,'y'), axis = 1)
    df_indi_d = df_indiTable.loc[df_indiTable['Alive']==False]
    df_indi_d['Age'] = df_indi_d.apply(lambda row: agecalc(row["Birth Date"],row["Death Date"],'y'), axis = 1)
    
    df_indi = pd.concat([df_indi_a, df_indi_d])
    for i in range(len(df_indi)): 
            if int(df_indi.loc[i,"Age"]) > 150:
                print("ERROR : Individual ID",str(df_indi.loc[i,'ID']),"has age more than 150 years")
            else:
                print("Individual ID",str(df_indi.loc[i,'ID']),"has age less than 150 years")
        

In [31]:
def us08(df_inditable, df_famtable):
    df_famtable
    df = df_famtable.loc[:,['ID','Married','Divorced','Children']].reset_index(drop=True)
    df.set_index(['ID','Married', 'Divorced'])

    df_e = df.explode('Children')
    df_c = df_e[df_e['Children'].notnull()]
    df_m = df_c.merge(df_inditable.loc[:,['ID','Birth Date']],
                 left_on='Children', right_on='ID',
                  how='left',suffixes=('','_Ind'))       
    df_m['MDiff'] = df_m.apply(lambda row: compareDate(row['Birth Date'],row['Married']),axis = 1)
    df_m['DDiff'] = df_m.apply(lambda row: agecalc(row['Divorced'],row['Birth Date'],'m'),axis = 1)
    for i in range(len(df_m)):
        if (df_m.loc[i,'MDiff'] == False) or (df_m.loc[i,'DDiff'] > 9):
            print("ERROR:Individual ID",df_m.loc[i,'Children']," has Birth before Marriage of Parents or after 9 months of Divorce")
        else:
            print("Individual ID",df_m.loc[i,'Children'],'has Birth date after Marriage of Parents or before 9 months of Divorce')

Load GEDCOM Files
Perform Data Processing to Tables 

In [32]:
file_path = r"C:\Users\sidha\Stevens\CS 555 - Agile Methods for Software Development\Project\Project 3\Sprint 4\Family tree-Sprint4-UAT.ged"
indiList,famList = pj3.gedcomRead(file_path)
df_IndiTable, df_FamTable = pj3.dataframe(indiList,famList)


Print the GEDCOM Data in tabular format

In [33]:
print(tabulate(df_IndiTable,headers = 'keys', tablefmt = 'psql'))
print(tabulate(df_FamTable,headers = 'keys', tablefmt = 'psql'))

+----+-------+----------------+-------+--------------+-------+---------+--------------+----------+---------+
|    | ID    | Name           | Sex   | Birth Date   |   Age | Alive   | Death Date   | Spouse   | Child   |
|----+-------+----------------+-------+--------------+-------+---------+--------------+----------+---------|
|  0 | @I1@  | Samay Singh    | M     | 1987 MAY 13  |    35 | True    | NA           | @F1@     | @F2@    |
|  1 | @I2@  | Satyajit Singh | M     | 1958 FEB 4   |    64 | True    | NA           | @F2@     | @F3@    |
|  2 | @I3@  | Susmita Sharma | F     | 1960 JAN 15  |    62 | True    | NA           | @F2@     | NA      |
|  3 | @I4@  | Sukumar Singh  | M     | 1934 DEC 14  |    87 | True    | NA           | @F3@     | NA      |
|  4 | @I5@  | Saswati Kumari | F     | 1935 NOV 30  |    86 | True    | NA           | @F3@     | NA      |
|  5 | @I6@  | Sujata Singh   | F     | 1962 JUL 8   |    59 | True    | NA           | @F6@     | @F3@    |
|  6 | @I7@  | Shee

USER STORIES 

In [34]:
print("************Story US01****************")
story_us01_refactor(df_IndiTable,df_FamTable)
print("************Story US02****************")
story_us02(df_IndiTable,df_FamTable)
print("************Story US03****************")
us03(df_IndiTable)
print("************Story US04****************")
us04(df_FamTable)
print("************Story US05****************")
us05(df_IndiTable,df_FamTable)
print("************Story US06****************")
us06(df_IndiTable,df_FamTable)
print("************Story US07****************")
us07(df_IndiTable)
print("************Story US08****************")
us08(df_IndiTable,df_FamTable)

************Story US01****************
Record @I1@  Check OK for Birth Dates 
Record @I1@ Check OK for Death Dates 
Record @I2@  Check OK for Birth Dates 
Record @I2@ Check OK for Death Dates 
Record @I3@  Check OK for Birth Dates 
Record @I3@ Check OK for Death Dates 
Record @I4@  Check OK for Birth Dates 
Record @I4@ Check OK for Death Dates 
Record @I5@  Check OK for Birth Dates 
Record @I5@ Check OK for Death Dates 
Record @I6@  Check OK for Birth Dates 
Record @I6@ Check OK for Death Dates 
Record @I7@  Check OK for Birth Dates 
Record @I7@ Check OK for Death Dates 
Record @I8@  Check OK for Birth Dates 
Record @I8@ Check OK for Death Dates 
Record @I9@  Check OK for Birth Dates 
Record @I9@ Check OK for Death Dates 
Record @I10@  Check OK for Birth Dates 
Record @I10@ Check OK for Death Dates 
Record @I11@  Check OK for Birth Dates 
Record @I11@ Check OK for Death Dates 
Record @I12@  Check OK for Birth Dates 
Record @I12@ Check OK for Death Dates 
Record @I13@  Check OK for Birt

C:\Users\sidha\AppData\Local\Temp\ipykernel_9204\2395536948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_indi_d['Date Check'] = df_indi_d.apply(lambda row : compareDate(row['Death Date'],row['Birth Date']),axis = 1)
C:\Users\sidha\AppData\Local\Temp\ipykernel_9204\717707603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_divorce['Date Check'] = df_divorce.apply(lambda row : compareDate(row['Divorced'],row['Married']),axis = 1)
C:\Users\sidha\AppData\Local\Temp\ipykernel_9204\2625196427.py:5